In [ ]:
import pandas as pd, geopandas as gpd
from rapidfuzz import process, fuzz
from tqdm import tqdm
from shapely import Point

In [ ]:
def parse_gps(lat_str, lon_str):
    """Parse GPS coordinates from Polish DMS format like "21*22'806" to decimal degrees"""
    if pd.isna(lat_str) or lat_str == '' or pd.isna(lon_str) or lon_str == '':
        return None
    try:
        # Try direct float conversion first (for decimal degrees)
        return float(lat_str), float(lon_str)
    except:
        try:
            # Parse degree*minute'second format like "21*22'806"
            if '*' in str(lat_str) and "'" in str(lat_str) and '*' in str(lon_str) and "'" in str(lon_str):
                lat_str, lon_str = str(lat_str).replace(',','').replace("''","'"), str(lon_str).replace(',','').replace("''","'")
                parts_lat, parts_lon = lat_str.split('*'), lon_str.split('*')
                min_sec_lat, min_sec_lon = parts_lat[1].split("'"), parts_lon[1].split("'")
                minutes_lat, minutes_lon = float(min_sec_lat[0]), float(min_sec_lon[0])
                seconds_lat, seconds_lon = float(min_sec_lat[0]), float(min_sec_lon[0])

                if minutes_lat > 59 or minutes_lon > 59 or seconds_lat >= 600 or seconds_lon >= 600:
                    return parse_as_decimal(lat_str), parse_as_decimal(lon_str)
                else:
                    return parse_as_degrees(lat_str), parse_as_degrees(lon_str)
            else:
                print(f"Error parsing coordinate '{lat_str} {lon_str}': Invalid format")  # Fixed undefined 'e'
                return None
        except Exception as e:
            print(f"Error parsing coordinate '{lat_str} {lon_str}': {e}")
            return None

def parse_as_decimal(coord_str):
    try:
        return float(coord_str)
    except:
        parts = coord_str.split('*', maxsplit=1)
        ms = parts[1].split("'", maxsplit=1)

        try:
            parts[0] = parts[0].replace("*", "0")
            ms[0] = ms[0].strip("*,`''").replace("''", "'").replace("*", "0")
            ms[1] = ms[1].strip("*,`''").replace("''", "'").replace("*", "0")
            ms[1] = 0 if ms[1] == '' else ms[1]
            if len(ms[1]) <= 2:
                div = 1
            else:
                div = 10
            return float(parts[0]) + float(ms[0])/100 + (float(ms[1])/div)/10000
        except Exception as e:
            print(f"Error parsing coordinate '{coord_str}': {e}")
            return None

def parse_as_degrees(coord_str):
    try:
        return float(coord_str)
    except:
        parts = coord_str.split('*', maxsplit=1)
        ms = parts[1].split("'", maxsplit=1)
        
        try:
            parts[0] = parts[0].replace("*", "0")
            ms[0] = ms[0].strip("*,`''").replace("''", "'").replace("*", "0")
            ms[1] = ms[1].strip("*,`''").replace("''", "'").replace("*", "0")
            ms[1] = 0 if ms[1] == '' else ms[1]

            if len(ms[1]) <= 2:
                div = 1
            elif len(ms[1]) == 3:
                div = 10
            elif len(ms[1]) == 4:
                div = 100
            else:
                div = 10

            return float(parts[0]) + float(ms[0])/60 + (float(ms[1])/div)/3600
        except Exception as e:
            print(f"Error parsing coordinate '{coord_str}': {e}")
            return None

def best_match(street, choices):
    """Return best fuzzy match + score from a list of choices"""
    if pd.isna(street) or not choices:
        return None, 0
    match, score, _ = process.extractOne(
        street, choices, scorer=fuzz.token_sort_ratio
    )
    return match, score

In [ ]:
gminy = gpd.read_file("gis/A03_Granice_gmin.shp", encoding='utf-8').to_crs(4326)
powiaty = gpd.read_file("gis/A02_Granice_powiatow.shp", encoding='utf-8').to_crs(4326)
wojewodztwa = gpd.read_file("gis/A01_Granice_wojewodztw.shp", encoding='utf-8').to_crs(4326)

In [ ]:
if False:
    osm_streets = gpd.read_file("poland_pbf/poland-250914.osm.pbf", layer="lines")

    osm_types = [
        'residential', 'tertiary', 'pedestrian', 'secondary',
        'living_street', 'primary', 'unclassified', 'service',
        'secondary_link', 'trunk', 'construction', 'primary_link',
        'motorway', 'tertiary_link', 'trunk_link', 'motorway_link']
    osm_streets = osm_streets[osm_streets["highway"].isin(osm_types)]

    osm_streets = osm_selected_streets[["name", "highway", "geometry"]].copy()
    osm_streets = osm_streets.sjoin(gminy[["JPT_NAZWA_", "JPT_KOD_JE", "geometry"]], how="left")
    osm_streets.drop(['index_right'], axis=1, errors="ignore").to_file('gis/roads_poland.shp')
else:
    osm_streets = gpd.read_file('gis/roads_poland.shp')


In [ ]:
osm_points = gpd.read_file("poland_pbf/poland-250914.osm.pbf", layer="points")
osm_polygons = gpd.read_file("poland_pbf/poland-250914.osm.pbf", layer="multipolygons")


In [ ]:
def extract_osm_attribute(df: pd.DataFrame, source_attr_name, target_attr_name):
    df[f"{source_attr_name}_loc"] = df["other_tags"].str.find(f'{source_attr_name}') + len(f'"{source_attr_name}"=>"')
    df[f"{source_attr_name}_loc_end"] = df.apply(lambda x: x["other_tags"].find('"', x[f'{source_attr_name}_loc']), axis=1)
    df[f"{target_attr_name}"] = df.apply(lambda x: x["other_tags"][x[f"{source_attr_name}_loc"]-1:x[f"{source_attr_name}_loc_end"]], axis=1)
    return df

if True:
    # EXTRACT ADDRESSES FROM POLYGONS (BUILDINGS)
    # osm_polygons = gpd.read_file("poland_pbf/poland-250914.osm.pbf", layer="multipolygons")
    addresses_b = osm_polygons.loc[(~osm_polygons["other_tags"].isna()) & (osm_polygons["other_tags"].str.contains("addr")), ["geometry", "other_tags"]]

    addresses_b["geometry"] = addresses_b.to_crs('+proj=cea').centroid.to_crs(addresses_b.crs)
    addresses_b = extract_osm_attribute(addresses_b, 'addr:housenumber', 'addr_num')
    addresses_b = extract_osm_attribute(addresses_b, 'addr:street', 'street_nm')
    addresses_b = addresses_b[["addr_num", "street_nm", "geometry"]]
    addresses_b = addresses_b.sjoin(gminy[["JPT_NAZWA_", "JPT_KOD_JE", "geometry"]], how="left")

    # EXTRACT ADDRESSES FROM POINTS
    # osm_points = gpd.read_file("poland_pbf/poland-250914.osm.pbf", layer="points")

    addresses_p = osm_points.loc[(~osm_points["other_tags"].isna()) & (osm_points["other_tags"].str.contains("addr")), ["geometry", "other_tags"]]

    addresses_p = extract_osm_attribute(addresses_p, 'addr:housenumber', 'addr_num')
    addresses_p = extract_osm_attribute(addresses_p, 'addr:street', 'street_nm')
    addresses_p = addresses_p[["addr_num", "street_nm", "geometry"]]
    addresses_p = addresses_p.sjoin(gminy[["JPT_NAZWA_", "JPT_KOD_JE", "geometry"]], how="left")

    # MERGE AND SAVE
    osm_adresses = pd.concat([addresses_b, addresses_p])
    osm_adresses.drop(['index_right'], axis=1, errors="ignore").to_file('gis/addr_poland.shp')
else:
    osm_adresses = gpd.read_file('gis/addr_poland.shp')

In [ ]:
accidents_df = pd.read_csv("csv/sewik_accidents.csv")

# Remove columns that start with "Unnamed:"
accidents_df = accidents_df.loc[:, ~accidents_df.columns.str.contains('^Unnamed')]

parse_gps_coords = True
if parse_gps_coords:
    coords = accidents_df.apply(lambda row: parse_gps(row['WSP_GPS_Y'], row['WSP_GPS_X']), axis=1)
    
    coord_list = coords.tolist()
    accidents_df['lat'] = pd.Series([coord[0] if coord is not None else None for coord in coord_list])
    accidents_df['lon'] = pd.Series([coord[1] if coord is not None else None for coord in coord_list])

    # Convert to numeric, coercing errors to NaN
    accidents_df['lon'] = pd.to_numeric(accidents_df['lon'], errors='coerce')
    accidents_df['lat'] = pd.to_numeric(accidents_df['lat'], errors='coerce')

accidents_df = gpd.GeoDataFrame(
    accidents_df, 
    geometry=gpd.GeoSeries.from_xy(
        accidents_df["lon"], accidents_df["lat"], crs="WGS84")
)
if False:
    # accidents_df.drop("geometry", axis=1).to_csv("csv/sewik_accidents.csv", index=False)

    accidents_df.loc[
        accidents_df.geometry.is_valid,
            [
                'ID', 'year', 'WOJ', 'WSP_GPS_X', 'WSP_GPS_Y', 'MIEJSCOWOSC' ,
                'ULICA_ADRES', 'NUMER_DOMU', "geometry"]
            ].to_file('gis/sewik_accidents_points.shp', driver='ESRI Shapefile')

In [ ]:
gminy["JPT_NAZWA_"] = gminy["JPT_NAZWA_"].astype(str).str.upper()
powiaty["JPT_NAZWA_"] = powiaty["JPT_NAZWA_"].astype(str).str.upper()
wojewodztwa["JPT_NAZWA_"] = wojewodztwa["JPT_NAZWA_"].astype(str).str.upper()

# Debug: Print initial counts
print(f"Initial gminy count: {len(gminy)}")
print(f"Initial powiaty count: {len(powiaty)}")
print(f"Initial wojewodztwa count: {len(wojewodztwa)}")

gminy["JPT_KOD_JE"] = gminy["JPT_KOD_JE"].astype(str)
gminy["KOD_POW"] = gminy["JPT_KOD_JE"].astype(str).str[:4]
gminy["KOD_WOJ"] = gminy["JPT_KOD_JE"].astype(str).str[:2]
powiaty["KOD_WOJ"] = powiaty["JPT_KOD_JE"].astype(str).str[:2]
powiaty["KOD_POW"] = powiaty["JPT_KOD_JE"].astype(str)

powiaty["JPT_KOD_JE"] = powiaty["JPT_KOD_JE"]
wojewodztwa["JPT_KOD_JE"] = wojewodztwa["JPT_KOD_JE"]

# Debug: Check for missing KOD_POW matches before merge
gminy_kod_pow = set(gminy["KOD_POW"].unique())
powiaty_kod_pow = set(powiaty["KOD_POW"].unique())
missing_pow_codes = gminy_kod_pow - powiaty_kod_pow
print(f"Unique KOD_POW in gminy: {len(gminy_kod_pow)}")
print(f"Unique KOD_POW in powiaty: {len(powiaty_kod_pow)}")
print(f"Missing KOD_POW codes in powiaty: {len(missing_pow_codes)}")
if missing_pow_codes:
    print(f"Missing codes sample: {list(missing_pow_codes)[:10]}")

# Use outer join to preserve all gminas and identify missing matches
gminy_powiaty = gminy[["JPT_KOD_JE", "JPT_NAZWA_", "KOD_POW"]].rename({"JPT_KOD_JE": "KOD_GMI", "JPT_NAZWA_": "GMINA"}, axis=1).merge(
    powiaty[["JPT_KOD_JE", "JPT_NAZWA_", "KOD_WOJ"]].rename({"JPT_NAZWA_": "POWIAT", "JPT_KOD_JE": "KOD_POW"}, axis=1),
    on="KOD_POW", how="left")

print(f"After first merge: {len(gminy_powiaty)}")
print(f"Records with missing POWIAT: {len(gminy_powiaty[gminy_powiaty['POWIAT'].isna()])}")

# Check for missing KOD_WOJ matches before second merge
gminy_kod_woj = set(gminy_powiaty["KOD_WOJ"].dropna().unique())
wojewodztwa_kod_woj = set(wojewodztwa["JPT_KOD_JE"].unique())
missing_woj_codes = gminy_kod_woj - wojewodztwa_kod_woj
print(f"Missing KOD_WOJ codes in wojewodztwa: {len(missing_woj_codes)}")

gminy_powiaty = gminy_powiaty.merge(
    wojewodztwa[["JPT_KOD_JE", "JPT_NAZWA_"]].rename({"JPT_NAZWA_": "WOJ", "JPT_KOD_JE": "KOD_WOJ"}, axis=1),
    on="KOD_WOJ", how="left")

print(f"After second merge: {len(gminy_powiaty)}")
print(f"Records with missing WOJ: {len(gminy_powiaty[gminy_powiaty['WOJ'].isna()])}")

# Show samples of missing data for debugging
if len(gminy_powiaty[gminy_powiaty["POWIAT"].isna()]) > 0:
    print("\nSample gminas without matching powiat:")
    print(gminy_powiaty[gminy_powiaty["POWIAT"].isna()][["GMINA", "KOD_POW"]].head())

# Modified condition to warn instead of raise exception
missing_powiaty = gminy_powiaty[gminy_powiaty["POWIAT"].isna()]
if len(missing_powiaty) > 0:
    print(f"WARNING: {len(missing_powiaty)} gminas could not be matched to powiaty")
    # You can choose to either filter them out or keep them with missing data
    # gminy_powiaty = gminy_powiaty.dropna(subset=["POWIAT"])  # Uncomment to remove unmatched records

powiaty_for_join = gminy_powiaty[~gminy_powiaty[["KOD_POW"]].duplicated(keep='first')][["KOD_POW", "POWIAT", "KOD_WOJ", "WOJ"]]
print(f"Final powiaty_for_join count: {len(powiaty_for_join)}")

gminy_obwarzanki = gminy_powiaty[gminy_powiaty[["WOJ", "POWIAT", "GMINA"]].duplicated(keep=False)]
gminy_obwarzanki.loc[gminy_obwarzanki["KOD_GMI"].str[-1]== '1', "GMINA"] = gminy_obwarzanki.loc[gminy_obwarzanki["KOD_GMI"].str[-1]== '1', "GMINA"] + " - OBSZAR WIEJSKI"
gminy_obwarzanki.loc[gminy_obwarzanki["KOD_GMI"].str[-1]== '2', "GMINA"] = gminy_obwarzanki.loc[gminy_obwarzanki["KOD_GMI"].str[-1]== '2', "GMINA"] + " - OBSZAR MIEJSKI"

gminy_powiaty = pd.concat([gminy_powiaty[~gminy_powiaty[["WOJ", "POWIAT", "GMINA"]].duplicated(keep=False)], gminy_obwarzanki])
print(f"Final gminy_powiaty count: {len(gminy_powiaty)}")

miasta_na_prawach_powiatu = gminy_powiaty.copy()
miasta_na_prawach_powiatu["GMINA"] = "POWIAT " + miasta_na_prawach_powiatu["GMINA"]
miasta_na_prawach_powiatu = miasta_na_prawach_powiatu[miasta_na_prawach_powiatu["POWIAT"]==miasta_na_prawach_powiatu["GMINA"]]

In [ ]:
# czyszczenie gmin
accident_gmina_correspondence = {
    'WARSZAWA BIAŁOŁĘKA': 'WARSZAWA',
    'WARSZAWA MOKOTÓW': 'WARSZAWA',
    'OSTROWICE': 'DRAWSKO POMORSKIE', # zlikwidowana w 2018 z powodu niewyplacalnosci
    'SITKÓWKA-NOWINY': 'NOWINY', # zmiana nazwy
    'JEDLNIA - LETNISKO': 'JEDLNIA-LETNISKO'
}


if "GMINA_org" not in accidents_df.columns:
    for old, new in accident_gmina_correspondence.items():
        accidents_df["GMINA_org"] = accidents_df["GMINA"].str.replace(old, new)
else:
    print ("Column GMINA_org already existing")
    for old, new in accident_gmina_correspondence.items():
        accidents_df["GMINA_org"] = accidents_df["GMINA_org"].str.replace(old, new)

# gminas_to_correct = accidents_df[["GMINA_org", "GMINA"]].drop_duplicates()
# gminas_to_correct = gminas_to_correct[~gminas_to_correct["GMINA"].duplicated(keep=False)]

accidents_df.loc[~accidents_df["GMINA_org"].isin(gminy_obwarzanki["GMINA"]), "GMINA"] = (
    accidents_df.loc[~accidents_df["GMINA_org"].isin(gminy_obwarzanki["GMINA"]), "GMINA_org"]
    .astype(str)
    .str.replace(" - OBSZAR WIEJSKI", "", regex=False)
    .str.replace(" - OBSZAR MIEJSKI", "", regex=False)
)

# accidents_df[(accidents_df["GMINA"].str.contains("KARPACZ")) & (~accidents_df["GMINA_org"].isin(gminy_obwarzanki["GMINA"]))]["GMINA"]

In [ ]:
# accidents_df[(accidents_df["GMINA"].str.contains("KARPACZ")) & (~accidents_df["GMINA_org"].isin(gminy_obwarzanki["GMINA"]))]["GMINA"]
acc_join = accidents_df.merge(gminy_powiaty, on=["WOJ", "POWIAT", "GMINA"], how="left")

acc_join_mnp = acc_join.loc[~acc_join["GMINA"].isin(gminy_powiaty["GMINA"])].drop(['KOD_POW', 'KOD_WOJ', 'GMINA', 'KOD_GMI'], axis=1).merge(miasta_na_prawach_powiatu, on=["WOJ", "POWIAT"], how="left")
acc_join_mnp["GMINA"] = acc_join_mnp["GMINA"].str.replace('POWIAT ', '')

acc_join_mnp_remaining = acc_join_mnp[~acc_join_mnp["GMINA"].isna()].copy()
acc_join_mnp_remaining["GMINA"] = acc_join_mnp_remaining["MIEJSCOWOSC"]
acc_join_mnp_remaining = acc_join_mnp_remaining.drop(['KOD_POW', 'KOD_WOJ', 'POWIAT', 'KOD_GMI'], axis=1).merge(gminy_powiaty, on=["WOJ", "GMINA"], how="left")

acc_join_mnp = acc_join_mnp[acc_join_mnp["GMINA"].isna()]

acc_join_mnp_remaining = pd.concat([
        acc_join_mnp_remaining[~(acc_join_mnp_remaining["ID"].duplicated(keep=False))],
        acc_join_mnp_remaining[(acc_join_mnp_remaining["ID"].duplicated(keep="last"))]
    ])

new_accidents_df = pd.concat([
        acc_join.loc[acc_join["GMINA"].isin(gminy_powiaty["GMINA"])],
        acc_join_mnp, acc_join_mnp_remaining
    ])

In [ ]:
accidents_df_sjoined = new_accidents_df.sjoin(gminy[["JPT_NAZWA_", "JPT_KOD_JE", "geometry"]], how="left")
name_matches = accidents_df_sjoined["JPT_NAZWA_"].str.upper().isin(
    accidents_df_sjoined["MIEJSCOWOSC"].str.upper())
accidents_df_sjoined[~name_matches]

In [ ]:
check = new_accidents_df[~new_accidents_df["ID"].isin(accidents_df_sjoined["ID"])]
if (len(check) > 0) or (len(new_accidents_df) != len(accidents_df_sjoined)) or len(new_accidents_df) != len(accidents_df):
    raise Exception("Some accidents were lost")

In [ ]:
accidents_df_sjoined["MIEJSCOWOSC"] = accidents_df_sjoined["MIEJSCOWOSC"].astype(str).str.upper()
osm_streets["JPT_NAZWA_"] = osm_streets["JPT_NAZWA_"].astype(str).str.upper()
osm_streets["name"] = osm_streets["name"].astype(str).str.upper()
osm_streets = osm_streets[osm_streets["name"]!='NONE']

In [ ]:
accidents_df_sjoined["MIEJSCOWOSC"]

In [ ]:
matches = []

# Ensure both dataframes have the JPT_NAZWA_ column in uppercase for matching
accidents_df_sjoined["JPT_NAZWA_"] = accidents_df_sjoined["JPT_NAZWA_"].str.upper()
osm_streets["JPT_NAZWA_"] = osm_streets["JPT_NAZWA_"].str.upper()

# Get unique municipalities from accidents data

accidents_to_correct = accidents_df_sjoined[
    ((accidents_df_sjoined["MIEJSCOWOSC"] == "WARSZAWA") | (accidents_df_sjoined["JPT_KOD_JE"] != accidents_df_sjoined["KOD_GMI"]))]

gminas_to_check = accidents_to_correct["KOD_GMI"].dropna().unique()

# Calculate total number of accidents to process
total_accidents = len(accidents_to_correct)

# Initialize progress bar with ETA
pbar = tqdm(total=total_accidents, desc="Processing accidents")

total_matches = 0
processed_count = 0
last_update = 0

# Process each municipality separately
for kod_gminy in gminas_to_check:
    # Get accidents in this municipality
    gmina = gminy_powiaty.loc[gminy_powiaty["KOD_GMI"] == kod_gminy, "GMINA"].iloc[0]
    accidents_in_municipality = accidents_to_correct[
        (accidents_to_correct["KOD_GMI"] == kod_gminy) &
        ((accidents_to_correct["GMINA"] == "WARSZAWA") | (accidents_to_correct["JPT_KOD_JE"] != accidents_to_correct["KOD_GMI"]))
    ]
    
    # Get streets in this municipality
    streets_in_municipality = osm_streets[osm_streets["JPT_KOD_JE"] == kod_gminy]
    adresses_in_municipality = osm_adresses[osm_adresses["JPT_KOD_JE"] == kod_gminy]
    
    if streets_in_municipality.empty:
        # Update progress bar for accidents we can't process
        pbar.update(len(accidents_in_municipality))
        processed_count += len(accidents_in_municipality)
        continue
    
    
    # Get unique street names for fuzzy matching (keep original case)
    street_names = streets_in_municipality["name"].dropna().unique().tolist()

    if not street_names:
        # Update progress bar for accidents we can't process
        pbar.update(len(accidents_in_municipality))
        processed_count += len(accidents_in_municipality)
        continue
    
    # Match each accident to the best street
    for idx, accident in accidents_in_municipality.iterrows():
        accident_street = accident["ULICA_ADRES"]
        accident_address = accident["NUMER_DOMU"]
        processed_count += 1
        
        if pd.isna(accident_street) or accident_street == '':
            # For missing street names, keep current coordinates and create a basic record
            if processed_count - last_update >= 50:
                pbar.update(processed_count - last_update)
                match_pct = (total_matches / processed_count * 100) if processed_count > 0 else 0
                pbar.set_postfix({
                    'matches': total_matches,
                    'match_rate': f'{match_pct:.1f}%'
                })
                last_update = processed_count
            
            # Create record with no street match but keep original coordinates
            match_record = {
                "accident_id": accident["ID"],
                "municipality": gmina,
                "accident_street": None,
                "matched_street": None,
                "similarity_score": None,
                "distance_to_street": None,
                "lat": accident["lat"],
                "lon": accident["lon"],
                "WSP_GPS_X": accident["WSP_GPS_X"],
                "WSP_GPS_Y": accident["WSP_GPS_Y"],
                "accident_geometry": accident["geometry"],
                "street_geometry": None
            }
            matches.append(match_record)
            total_matches += 1
            continue

        # Find best matching street name (case-insensitive)
        best_street, similarity_score = best_match(accident_street, street_names)

        # Lower minimum threshold but add better validation
        if best_street and similarity_score >= 60:  # Reduced from 75 to 60            
            # Get ALL street segments with this name (case-insensitive search)
            matched_streets = streets_in_municipality[
                streets_in_municipality["name"].str.upper() == str(best_street).upper()]

            # Find the closest street segment geometrically
            # Find the closest street segment geometrically
            accident_point = accident["geometry"]
            best_distance = float('inf')
            closest_street = None
            best_coords = None
            
            # First, let's parse coordinates once for this accident
            coord_y_clean = str(accident["WSP_GPS_Y"]).replace("''","'").replace(',','')
            coord_x_clean = str(accident["WSP_GPS_X"]).replace("''","'").replace(',','')
            
            decimal_lat = parse_as_decimal(coord_y_clean)
            decimal_lon = parse_as_decimal(coord_x_clean)
            degree_lat = parse_as_degrees(coord_y_clean)
            degree_lon = parse_as_degrees(coord_x_clean)
            
            # Test which coordinate parsing gives better results
            candidate_points = []
            if decimal_lat is not None and decimal_lon is not None:
                candidate_points.append({"point": Point(decimal_lon, decimal_lat), "lat": decimal_lat, "lon": decimal_lon, "type": "decimal"})
            if degree_lat is not None and degree_lon is not None:
                candidate_points.append({"point": Point(degree_lon, degree_lat), "lat": degree_lat, "lon": degree_lon, "type": "degree"})
            if decimal_lat is not None and degree_lon is not None:
                candidate_points.append({"point": Point(degree_lon, decimal_lat), "lat": decimal_lat, "lon": degree_lon, "type": "decimal_lat_degree_lon"})
            if degree_lat is not None and decimal_lon is not None:
                candidate_points.append({"point": Point(decimal_lon, degree_lat), "lat": degree_lat, "lon": decimal_lon, "type": "degree_lat_decimal_lon"})

            # Add original coordinates if available
            if not pd.isna(accident["lat"]) and not pd.isna(accident["lon"]):
                candidate_points.append({"point": Point(accident["lon"], accident["lat"]), "lat": accident["lat"], "lon": accident["lon"], "type": "original"})
            
            for _, street_segment in matched_streets.iterrows():
                # Test all coordinate interpretations against this street segment
                for coord_candidate in candidate_points:
                    distance = coord_candidate["point"].distance(street_segment["geometry"])
                    
                    if distance < best_distance:
                        best_distance = distance
                        best_coords = coord_candidate
                        closest_street = street_segment
            
            # Convert distance to meters for validation (approximate for Poland ~52°N)
            distance_meters = best_distance * 111320
            
            # Only accept matches if they're within reasonable distance (e.g., 500 meters)
            # AND have good similarity score, OR very close distance with moderate similarity
            is_valid_match = False
            
            if distance_meters <= 100 and similarity_score >= 60:  # Very close, moderate similarity OK
                is_valid_match = True
            elif distance_meters <= 200 and similarity_score >= 70:  # Close, good similarity
                is_valid_match = True
            elif distance_meters <= 500 and similarity_score >= 80:  # Moderate distance, high similarity
                is_valid_match = True
            elif similarity_score > 95:  # Almost same
                is_valid_match = True
            match_record = {
                "accident_id": accident["ID"],
                "municipality": gmina,
                "accident_street": accident_street,
                "matched_street": best_street,
                "similarity_score": similarity_score,
                "distance_to_street": best_distance,
                "distance_to_street_meters": distance_meters,
                "WSP_GPS_X": accident["WSP_GPS_X"],
                "WSP_GPS_Y": accident["WSP_GPS_Y"],
                "best_lat": None,
                "best_lon": None,
                "best_coord_type": None,
                "lat": accident["lat"],
                "lon": accident["lon"],
                "accident_geometry": accident["geometry"],
                "street_geometry": None
            }

            if is_valid_match and closest_street is not None:
                match_record["best_lat"] = best_coords["lat"]
                match_record["best_lon"] = best_coords["lon"]
                match_record["best_coord_type"] = best_coords["type"]
                match_record["street_geometry"] = closest_street["geometry"]
                total_matches += 1
                
            matches.append(match_record)

        # Only update progress bar every 50 records
        if processed_count - last_update >= 50:
            pbar.update(processed_count - last_update)
            match_pct = (total_matches / processed_count * 100) if processed_count > 0 else 0
            pbar.set_postfix({
                'matches': total_matches,
                'match_rate': f'{match_pct:.1f}%'
            })
            last_update = processed_count

# Final update for any remaining records
if processed_count > last_update:
    pbar.update(processed_count - last_update)
    match_pct = (total_matches / processed_count * 100) if processed_count > 0 else 0
    pbar.set_postfix({
        'matches': total_matches,
        'match_rate': f'{match_pct:.1f}%'
    })

pbar.close()

# Create final dataframe
final_df = pd.DataFrame(matches)

# Convert distance from degrees to meters (approximate conversion at Poland's latitude)
# 1 degree ≈ 111,320 meters at the equator, but we need to account for latitude
# For Poland (around 52°N), 1 degree longitude ≈ 69,470 meters, 1 degree latitude ≈ 111,320 meters
# We'll use an average approximation for the distance calculation
if len(final_df) > 0:
    # Convert distance from degrees to meters (approximate conversion for Poland's latitude ~52°N)
    # Using Haversine approximation: 1 degree ≈ 111,320 meters
    final_df['distance_to_street_meters'] = final_df['distance_to_street'] * 111320
    
    # Round to 1 decimal place for readability
    final_df['distance_to_street_meters'] = final_df['distance_to_street_meters'].round(1)

print(f"\n" + "="*50)
print(f"FINAL RESULTS:")
print(f"Total matches created: {len(final_df)}")
if len(final_df) > 0:
    print(f"Similarity score distribution:")
    print(final_df["similarity_score"].describe())
    
    print(f"\nDistance to street distribution (meters):")
    print(final_df["distance_to_street_meters"].describe())
    
    # Filter for high-quality matches
    high_quality_matches = final_df[final_df["similarity_score"] >= 80]
    print(f"High quality matches (>= 80% similarity): {len(high_quality_matches)}")
    print(f"Medium quality matches (70-79% similarity): {len(final_df) - len(high_quality_matches)}")
    
    # Show closest matches
    closest_matches = final_df.nsmallest(10, 'distance_to_street_meters')
    print(f"\n10 closest matches:")
    print(closest_matches[['accident_id', 'accident_street', 'matched_street', 'similarity_score', 'distance_to_street_meters']])

In [ ]:
final_df = pd.DataFrame(matches)

final_df.to_csv("gis/street_matches_v3.tsv", sep="\t")

In [ ]:
updated_latlon = final_df[["accident_id", "best_lat", "best_lon"]].rename({"accident_id": "ID", "best_lat": "lat", "best_lon": "lon"}, axis=1).dropna().set_index("ID")
updated_latlon = updated_latlon[~updated_latlon.index.duplicated()]
accidents_df = accidents_df.set_index("ID")
accidents_df.update(updated_latlon)

In [ ]:
accidents_df.reset_index().drop("geometry", axis=1).to_csv("csv/sewik_accidents_v3.csv", index=False)

In [ ]:
accidents_df = gpd.GeoDataFrame(
    accidents_df, 
    geometry=gpd.GeoSeries.from_xy(
        accidents_df["lon"], accidents_df["lat"], crs="WGS84"))

accidents_df.reset_index().loc[
    accidents_df.reset_index().geometry.is_valid,
        [
            'ID', 'year', 'WOJ', 'WSP_GPS_X', 'WSP_GPS_Y', 'MIEJSCOWOSC' ,
            'ULICA_ADRES', 'NUMER_DOMU', "geometry"]
        ].to_file('gis/sewik_accidents_points_v3.shp', driver='ESRI Shapefile')